In [1]:
import numpy as np
import numpy.ma as ma


import sdfitsio as sdf
from importlib import reload
reload(sdf);

Create a masked array 'a' with a typical LMT structure

In [2]:
ntime = 1000
nbeam = 16
npol = 2
nband = 6
nchan = 256

adim=(ntime,nbeam,npol,nband,nchan)
%time a = sdf.gen_data(adim)

print('Size of data',sdf.dimsize(adim))

CPU times: user 2.83 s, sys: 104 ms, total: 2.93 s
Wall time: 2.93 s
Size of data 49152000


Write it as SDFITS

In [3]:
%time sdf.my_write_sdfits('junk.fits',a)

SHAPE= (1000, 16, 2, 6, 256)
NAXIS2= 192000
NCHAN= 256
(192000,)
(192000, 256)
Written junk.fits
CPU times: user 934 ms, sys: 5.01 s, total: 5.94 s
Wall time: 6.01 s


Read it back in


In [4]:
%time sdf.my_read('junk.fits')

File:      junk.fits
Size:      4 cols x 192000 rows
Telescope: LMT/GTM
Object:    ['NOISE']
DateObs:   0.0
DIMS2: (1000, 16, 2, 6, 256)
(192000, 256)
(1000, 16, 2, 6, 256)
Data.sum() =  -4923621.5
CPU times: user 285 ms, sys: 33.8 ms, total: 319 ms
Wall time: 313 ms


masked_array(
  data=[[[[[0.29746973514556885, -0.19666922092437744,
            -0.6603138446807861, ..., 2.727635622024536,
            0.5510263442993164, 0.0013596965000033379],
           [-2.207197427749634, 0.017874429002404213,
            -0.35693684220314026, ..., -0.7900201082229614,
            -0.5634557008743286, -1.288738489151001],
           [-1.668457269668579, 0.6568979620933533,
            0.6072496175765991, ..., 0.6622494459152222,
            0.6430400609970093, -0.3295194208621979],
           [-0.9131069183349609, -1.1924545764923096,
            0.17407244443893433, ..., -0.10995909571647644,
            1.3590916395187378, 1.904985785484314],
           [-0.650054395198822, 0.38570261001586914,
            0.21386289596557617, ..., -0.8403933644294739,
            -0.7441658973693848, 0.5233352780342102],
           [-1.0970683097839355, -0.5036274194717407,
            -0.24536831676959991, ..., 0.15637952089309692,
            -1.3566079139709473, -0.90922

Set up a masking operation. Low level method.

In [5]:
mask = {}
mask[0] = '0:110'     # time 
#mask[1] = '3'        # beam
#mask[2] = '0'        # pol
#mask[3] = '4'        # band
mask[4] = '81:83'    # chan


In [6]:
%time sdf.data_mask(a, mask)


DATA_MASK:  {0: '0:110', 4: '81:83'}
CMD: data.mask[0:110,:,:,:,81:83] = True
CPU times: user 1.69 ms, sys: 1.75 ms, total: 3.43 ms
Wall time: 1.68 ms


Report what fraction was masked

In [7]:
%time sdf.data_masked(a)

data_masked: 42240 / 49152000 = 0.0859375%
CPU times: user 146 ms, sys: 9.51 ms, total: 155 ms
Wall time: 154 ms


In [8]:
%time sdf.my_write_sdfits('junk_masked.fits',a)

SHAPE= (1000, 16, 2, 6, 256)
NAXIS2= 192000
NCHAN= 256
(192000,)
(192000, 256)
Written junk_masked.fits
CPU times: user 645 ms, sys: 5.07 s, total: 5.72 s
Wall time: 5.8 s


Read back the original unmasked

In [9]:
%time b=sdf.my_read('junk.fits')

File:      junk.fits
Size:      4 cols x 192000 rows
Telescope: LMT/GTM
Object:    ['NOISE']
DateObs:   0.0
DIMS2: (1000, 16, 2, 6, 256)
(192000, 256)
(1000, 16, 2, 6, 256)
Data.sum() =  -4923621.5
CPU times: user 289 ms, sys: 24.5 ms, total: 314 ms
Wall time: 308 ms


In [10]:
b.shape

(1000, 16, 2, 6, 256)

In [11]:
%time sdf.data_masked(b)

data_masked: 0 / 49152000 = 0%
CPU times: user 138 ms, sys: 11.7 ms, total: 150 ms
Wall time: 152 ms


In [12]:
%time c = sdf.my_read('junk_masked.fits')

File:      junk_masked.fits
Size:      4 cols x 192000 rows
Telescope: LMT/GTM
Object:    ['NOISE']
DateObs:   0.0
DIMS2: (1000, 16, 2, 6, 256)
(192000, 256)
(1000, 16, 2, 6, 256)
Data.sum() =  nan
CPU times: user 335 ms, sys: 17.8 ms, total: 353 ms
Wall time: 358 ms


In [13]:
%time sdf.data_masked(c)

data_masked: 42240 / 49152000 = 0.0859375%
CPU times: user 141 ms, sys: 9.82 ms, total: 151 ms
Wall time: 149 ms


Indeed, the masking have now been applied (as NaN) in the SDFITS file. masks cannot be undone here.